<a href="https://colab.research.google.com/github/lemer914/Clustering-DNA/blob/main/LSH_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## DNA Data Storage
Recovering the data stored on DNA by clustering the noisy reads.

In [ ]:
!pip install biopython
!pip install sklearn
!pip install scikit-bio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 22.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post7-py3-none-any.whl size=2951 sha256=601ab847cc4b29aa9c069c984870edcfdf4cecf570dcfa11d973050143b32e11
  Stored in directory: /root/.cache/pip/wheels/c8/9c/85/72901eb50bc4bc6e3b2629378d172384ea3dfd19759c77fd2c
Successfully built sklearn
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 28.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 69.5 MB/s eta 0:00:00
  Created wheel for scikit-bio: filename=scikit_bio-0.5.9-cp310-cp310-linux_x86_64.whl size=2469057 sha256=e79e83c050a38

In [ ]:
import numpy as np
import random as randy
from random import shuffle
from Bio.Align.Applications import MuscleCommandline
from Bio import AlignIO
from skbio.alignment import local_pairwise_align_ssw
from skbio import DNA
import itertools
import operator
import time
import zipfile
import subprocess as sp
import hashlib
# from include import *
from random import random

## Data preparation

### Note: I commented all of the following code out to implement my own data preparation
You may choose to work with either a simulated dataset or a real synthesized dataset. The overall steps you need to take for each data are as follows:<br> <br>
Simulated data: <br>
In this example, the following stages are tested: <br>
1- Loading File_1.zip and encoding it using a Reed-Solomon encoder. <br>
2- Generating noisy reads from the encoded file. <br>
3- Clustering the reads using either the Trivial clustering or the Locality-Sensitive Hashing (LSH) method. <br>
4- Generating candidates from the clusters by performing multiple sequence alignment and majority voting. <br>
5- Putting the candidates into a Reed-Solomon decoder and recovering the oiginal data. <br>
*Encoding note: <br>
Note that n, k, N, K, nus, and numblocks (this one is printed out when the following function is called) will be needed for decoding. <br> <br>
Real synthesized data: <br>
1- Load files File1_ODNA.txt and I16_S2_R1_001.fastq which contain the original sequences and the noisy reads, respectively. <br>
2- Clustering the reads using either the Trivial clustering or the Locality-Sensitive Hashing (LSH) method. <br>
3- Generating candidates from the clusters by performing multiple sequence alignment and majority voting. <br>
4- Putting the candidates into a Reed-Solomon decoder and recovering the oiginal data. <br>

In [ ]:
# def encode(infile):
#     stat = sp.Popen(["./simulate/texttodna", "--n=16383", "--k=10977", "--N=20", "--K=18", "--nuss=6", "--l=4", \
#                      "--encode", infile,"--output=./data/data_encoded.txt"],stdout = sp.PIPE)
#     for line in stat.stdout.read().splitlines():
#         print(line)
#     return True

In [ ]:
# def generate_noisy_reads(infile,p,n):
#     # infile: encoded file
#     # p: deletion probabilty per character (this increases per character index to mimick the synthesis procedure)
#     # n: number of noisy reads per original sequence

#     orig_seqs = []
#     f = open(infile,"r")
#     for seq in f.read().splitlines():
#         orig_seqs += [seq]


#     rand_replace = lambda c,d: c if random() > p*d else '' # this function induces deletions

#     population = []
#     for seq in orig_seqs:
#         for i in range(n):
#             s = ''.join([rand_replace(c,j) for j,c in enumerate(seq)])
#             population += [s]
#     shuffle(population)
#     return orig_seqs,population

In [ ]:
# SIMULATE = True

In [ ]:
# if SIMULATE:
#     ##### Encode
#     s = time.time()
#     encode("--input=./data/File_1.zip")
#     print("Encoding time:",round(time.time()-s,1),"s")
#     ##### Generate noisy reads by inducing deletions (perturbation step)
#     orig_seqs,reads = generate_noisy_reads("./data/data_encoded.txt",0.0005,6)
# else:
#     ##### Load synthesized data
#     orig_seqs = file_to_list("data/File1_ODNA.txt")
#     filename = "data/I16_S2_R1_001.fastq"
#     seqs = fastq_to_list(filename)
#     print("all sequences: ", len(seqs))
#     print("all orig sequences: ", len(orig_seqs))
#     reads = [seq for seq in seqs if len(seq) >= 55 and len(seq)<=70]
#     print("all trimmed sequences: ",  len(reads))

In [ ]:
# This function is used if you want to read sequences from a file instead of generating them
# Reads from a file of clusters, where each cluster is separated by a line of '='
# Returns a list of lists, where each list in the returned list is a cluster of sequences. This list of list contains the underlying clusters
def read_file():
    from google.colab import files
    uploaded = files.upload()
    microsoft_seqs = list(uploaded.keys())[0]
    file = open(microsoft_seqs, "r")
    number_clusters = 0
    current_cluster = []
    underlying_clusters = []
    reads = []
    for line in file:
        if number_clusters == 500:
            break
        if '=' in line:
          if current_cluster:
            number_clusters += 1
            underlying_clusters.append(current_cluster + [])
            current_cluster.clear()
        else:
            current_cluster.append(line.strip())
            reads.append(line.strip())
    file.close()
    return reads, underlying_clusters

Generating random strings and then creating noisy copies

In [ ]:
# Constants used in the data generation process
NUMBER_OF_REFERENCES = 500 # Number of original sequences of DNA that noisy copies will be made of
REFERENCE_LENGTH = 150 # the length of each original reference
# For each reference, the number of copies to be generated will come from N(mean, var)
MEAN_COPIES = 10.0
STAND_DEV_COPIES = 2.0 # note this is the standard deviation, not the variance
# The following are the probabilities of each kind of error occurring when making a copy of a reference string
INSERTION_PROB = 0.017
DELETION_PROB = 0.02
SUB_PROB = 0.022
MATCH_PROB = 1 - (INSERTION_PROB + DELETION_PROB + SUB_PROB) # probability of no error occurring for a given letter
# If testing outlier robustness, outlier strands are generated using the following error rate
OUTLIER_ERROR = 0.2 # this is epsilon prime in the billions paper

In [ ]:
# These functions are used if you want to generate sequences instead of read them from a file

# Generates all of our random reference strings, returns as a list
def construct_references():
    references = [] # list of all reference sequences
    current_ref = [] # the current reference
    for _ in range(NUMBER_OF_REFERENCES):
        for _ in range(REFERENCE_LENGTH):
            current_ref.append(ALPHABET[randy.randint(0, 3)]) # choose a random letter to add
        references.append("".join(current_ref)) # join all letters of our reference into a string, and add to list of references
        current_ref.clear() # clear current reference so we can have an empty string for next iteration
    return references

# For each reference, noisy copies are constructed
# Returns a list of lists, where each list in the returned list is a cluster of sequences
# This list of list contains the underlying clusters
def construct_copies(references):
    all_copies = [] # list of lists of copies
    reference_copies = [] # list of all copies of a given reference
    for ref in references:
        number_of_copies = int(np.random.normal(loc=MEAN_COPIES, scale=STAND_DEV_COPIES, size=None))
        # number_of_copies = 10 # the above line is the number of copies we actually want
        if number_of_copies > 0:
            for _ in range(number_of_copies):
                current_copy = []
                for letter in ref:
                    roll_die = True # our boolean to see if we should calculate the prob again
                    while roll_die:
                        # get random value to determine which type of error (or no error) occurs
                        rand = randy.randint(1, 1000) / 1000
                        # insertions
                        if rand <= INSERTION_PROB:
                            current_copy.append(ALPHABET[randy.randint(0, 3)]) # choose a random letter to add
                        # substitutions
                        elif rand <= INSERTION_PROB + SUB_PROB:
                            new_letter = ALPHABET[randy.randint(0, 3)] # choose a random new letter to add
                            while new_letter == letter:
                                new_letter = ALPHABET[randy.randint(0, 3)] # again, choose a random new letter to add
                            current_copy.append(new_letter) # add the new letter
                            roll_die = False
                        # matches
                        elif rand <= INSERTION_PROB + SUB_PROB + MATCH_PROB:
                            current_copy.append(letter) # add the letter we are on
                            roll_die = False
                        # deletions, we don't append the letter
                        else:
                            roll_die = False
                reference_copies.append("".join(current_copy))
                current_copy.clear()
        all_copies.append(reference_copies + [])
        reference_copies.clear()
    return all_copies
    # print(all_copies)

# This function can be used to turn our list of lists (i.e. the underlying clustering) into a list of all of our sequences that we can perform
# the clustering algorithm on
# Takes in a list of lists and flattens it into a list
def flatten(lst):
    return [item for sublist in lst for item in sublist]

In [ ]:
# This function also generated random data, but it uses very low error rates which are characteristic of certain sequencing methods
def generate_without_file(p, n):
    # p: deletion probabilty per character (this increases per character index to mimick the synthesis procedure)
    # n: number of noisy reads per original sequence

    orig_seqs = construct_references()

    from random import random
    rand_replace = lambda c,d: c if random() > p*d else '' # this function induces deletions

    population = []
    current_cluster = []
    underlying_clustering = []
    for seq in orig_seqs:
        for i in range(n):
            s = ''.join([rand_replace(c,j) for j,c in enumerate(seq)])
            population += [s]
            current_cluster += [s]
        underlying_clustering.append(current_cluster + [])
        current_cluster.clear()
    shuffle(population)
    return underlying_clustering,population

## Clustering
You may choose to work with one of the following clustering methods: <br>
1. Trivial clustering: looking at the first "nbeg" characters of each sequence to cluster the reads. This option is way faster than the other one, especially if you choose to work with the syntesized dataset. This is important since choosing this option reducing runtime of the following sections. <br>
2. Locality-Sensititve Hashing (LSH): clustering the reads based on the computed LSH signatures. If this option is chosen when working on the synthesized dataset, both clusering and alignment stages will be slower than the trivial clustering.

In [ ]:
#===== assign numbers to shingles of each sequence=====#
def kmerDNA(seq,k=3):
    kmer = []
    for ell in range(len(seq)-k+1):
        nstr = seq[ell:ell+k]
        index = 0
        for j,c in enumerate(nstr):
            if c == 'A':
                i = 0
            elif c == 'C':
                i = 1
            elif c == 'G':
                i = 2
            elif c == 'T':
                i = 3
            else:
                index = -1
                break
            index += i*(4**j)
        kmer += [index]
    # print(kmer)
    return kmer
#=====min-hash object=====#
class minhashsig():
    # min-hash of k-mers
    def __init__(self,m,k):
        # m is the number of signatures
        self.tables = [np.random.permutation(4**k) for i in range(m)]
        self.k = k
    def generate_signature(self,seq):
        kmer = kmerDNA(seq,self.k)
        # my code below this, I kept getting a Val error but I fixed it
        sig = []
        table_vals = []
        number = 0
        for table in self.tables:
          for i in kmer:
            table_vals.append(table[i])
          try:
            sig.append(min(table_vals))
            number += 1
          except:
            print('Val error')
            # print(kmer)
            print('number of k-mers:', number)
          table_vals.clear()
        # try:
        #   sig = [ min([table[i] for i in kmer]) for table in self.tables]
        # except:
        #   print('Value Error')
        #   print(kmer)
        #   for table in self.tables:
        #     print(table)
        return sig
#=====pair detection=====#
def extract_similar_pairs(sigs,m,k_lsh,ell_lsh,maxsig):
    # sigs: minhash signatures
    # ell_lsh: number of LSH signatures
    # k_lsh: number of MH signatures to be concatenated
    # we use generatrs to yield a number of pairs at a time for the sake of memory efficiency

    pairs = set([])

    # generate ell_lsh random indices
    for ell in range(ell_lsh):
        pair_count = 0
        s = time.time()
        lshinds = np.random.permutation(m)[:k_lsh]
        # generate LSh signatures
        lshsigs = []
        for sig in sigs:
            lshsig = 0
            for i,lshind in enumerate(lshinds):
                lshsig += sig[lshind]*(maxsig**i)
            lshsigs += [lshsig]
        d = {}
        for ind,sig in enumerate(lshsigs):
            if sig in d:
                d[sig] += [ind]
            else:
                d[sig] = [ind]
        for candidates in d.values():
            cent = set([])
            if len(candidates) > 1:
                for pair in itertools.combinations(candidates,2):
                    cent.add(pair[0])
                    if len(cent)==1:
                        pairs.add(pair)
                    else:
                        break

        yield pairs,ell
        pair_count += len(pairs)
        pairs = set([])
#=====form clusters based on pairs=====#
def center_cluster(pairs):
    clusters = {}
    hold = 0
    t_counter = 0
    ell_copy = 0
    pairsize = 0
    while not hold:

        try:
            out = next(pairs)
            pairs_sort = list(out[0])
            ell = out[1]
            pairsize += len(pairs_sort)
            pairs_sort.sort()
            s = time.time()
            for (u,v) in pairs_sort:
                if u in clusters:
                    clusters[u] += [v]

                if v in clusters:
                    clusters[v] += [u]

                if v not in clusters and u not in clusters:
                    clusters[u] = [v]

        except StopIteration:
            hold = 1
            print("clustering completed","---",pairsize,"pairs clustered")
        if ell==ell_copy:
            t_counter += time.time()-s
        else:
            print("Clustering time for LSH",ell_copy,":",t_counter,'\n')
            t_counter = time.time()-s
            ell_copy = ell

    return clusters

#=====LSH clustering (main function)=====#
def lsh_cluster(seqs,m,k,k_lsh=2,ell_lsh=4):
    # This is the main function
    maxsig = 4**k
    minhash = minhashsig(m,k)
    # for i in range(20):
    #   print(len(seqs[i]))
    #   print(seqs[i])
    #   print('+++++++++')
    sigs = [minhash.generate_signature(seq[:40]) for seq in seqs]
    pairs = extract_similar_pairs(sigs,m,k_lsh,ell_lsh,maxsig)
    clusters = center_cluster(pairs)
    return clusters

In [ ]:
def filter_nonunique(seqs):
    # filter out sequences that appear many times
    d = {}
    ctr = 0
    for i,seq in enumerate(seqs):
        if seq in d:
            d[seq] += [i]
        else:
            d[seq] = [i]
    import operator
    sorted_d = sorted(d.items(), key=operator.itemgetter(1))
    sorted_d.reverse()
    return d,ctr

In [ ]:
TRIVIAL = False # when real synthesized data is selected, set this flag to "True" (this dramatically reduces the runtime)

In [ ]:
# There are three kinds of data that you could try to cluster
# Type 1 is the Microsoft data from the file
# Type 2 is generated data using the experimental error rates (insertion rate = 0.17, etc.)
# Type 3 is generated data using very low error, like the authors of the LSH algo paper used
# You can uncomment whichever type you want to use, and comment out the others

# Type 1
# reads, underlying = read_file()

# Type 2
references = construct_references()
underlying = construct_copies(references)
reads = flatten(underlying)

# Type 3
# underlying, reads = generate_without_file(0.0005, 10) # 0.0005, 6
# print(underlying[0])
# print(reads[0])

In [ ]:


if TRIVIAL:
    start = time.time()
    nbeg=14
    # I added this
    reads = read_file()
    d,ctr = filter_nonunique([seq[:nbeg] for seq in reads])
    clusters = [d[a] for a in d if len(d[a]) > 3]
    end = time.time()
    # print("Runtime:",round(end-start,1),"s")
    # print(len(clusters),"number of clusters created.")
    fclusts = clusters.copy()
else:
    # set up the parameters and call the lsh_cluster function
    k_lsh = 2 # 4 was the given value in their code
    sim = 0.01 # 0.5 was the given value in their code
    ell_lsh = int(1/(sim**k_lsh))
    m,k=50,5 # 50,5 were the given values in their code
    # I added this
    start = time.time()
    clusters = lsh_cluster(reads,m,k,k_lsh,ell_lsh)
    end = time.time()

    print("Runtime:",round(end-start,1),"s")
    print(len(clusters),"number of clusters created")

Streaming output truncated to the last 5000 lines.

Clustering time for LSH 37501 : 0.003252744674682617 

Clustering time for LSH 37502 : 0.004208087921142578 

Clustering time for LSH 37503 : 0.0048084259033203125 

Clustering time for LSH 37504 : 0.0033762454986572266 

Clustering time for LSH 37505 : 0.004002571105957031 

Clustering time for LSH 37506 : 0.005909442901611328 

Clustering time for LSH 37507 : 0.003968000411987305 

Clustering time for LSH 37508 : 0.007604837417602539 

Clustering time for LSH 37509 : 0.004695892333984375 

Clustering time for LSH 37510 : 0.003055572509765625 

Clustering time for LSH 37511 : 0.0034062862396240234 

Clustering time for LSH 37512 : 0.003086566925048828 

Clustering time for LSH 37513 : 0.0032701492309570312 

Clustering time for LSH 37514 : 0.003091096878051758 

Clustering time for LSH 37515 : 0.004113912582397461 

Clustering time for LSH 37516 : 0.00496983528137207 

Clustering time for LSH 37517 : 0.004593372344970703 

Clustering

## Filtering the clusters (do this step only if you have chosen LSH clustering)
In this stage, we use the max_match function to filter the clusters. This function checks the simlarity of each cluster member and its cluster center based on local alignment.

In [ ]:
# adding the center of each cluter to the cluster (also removing duplicates from each cluster)
clusts = [ [c] + list(set(clusters[c])) for c in clusters if len(clusters[c]) > 3]

In [ ]:
#=====max matching=====#
def max_match(seq1,seq2):
    # This function checks whether seq1 and seq2 are similar or not
    # Checking all pairs within a cluster dramatically increases the time complexity,
    # so by default, in the next cell, we call this function to only check the pairs
    # that one of their members is the cluster center

    alignment,score,start_end_positions \
        = local_pairwise_align_ssw(DNA(seq1) , DNA(seq2) , match_score=2,mismatch_score=-3)
    a = str(alignment[0])
    b = str(alignment[1])
    ctr = 0
    for i,j in zip(a,b):
        if i==j:
            ctr += 1
    return ctr

In [ ]:
th = 50 # filtering threshold, 35 was the given value in their code

k = len(clusts)
s = time.time()
fclusts = []
for i,c in enumerate(clusts):
    cent = reads[c[0]]
    cc = [c[0]]
    for e in c[1:]:
        score = max_match(cent,reads[e])
        if score >= th:
            cc += [e]
    fclusts += [cc]
    if i%1000 == 0:
        print("%",round(i*100/len(clusts),2),"of the clusters are filtered.")
print("filtering time for",k,"clusters:",round(time.time()-s,2),"s")

##Implementing our simple accuracy measurement

For the q-gram algorithm I used a simple accuracy measurement like this, where an indicator variable is a one if for any cluster in the underlying clustering, there is a cluster that is the same as that in our clustering (this is the case if gamma = 1.0).

In [ ]:
import random
# print(fclusts[:10])
all_seqs = reads + []
our_clusters_numbered = fclusts + []
our_clusters_lettered = []
for i, clusterinds in enumerate(our_clusters_numbered):
    cluster = []
    for i in clusterinds:
      if reads[i] in all_seqs:
        all_seqs.remove(reads[i])
        cluster.append(reads[i])
    # cluster = [reads[i] for i in clusterinds if reads[i] not in counted]
    our_clusters_lettered.append(cluster + [])
    cluster.clear()
# random.shuffle(our_clusters_lettered)
print(our_clusters_lettered[:5])
print(underlying[:2])
# Sometimes reads are duplicated in the process, so this sees how many times that happened
print('length of all seqs left over:', len(all_seqs))

# Filter out the duplicate reads
our_clusters_new = []
our_clusters_as_sets = []
for clustery in our_clusters_lettered:
  if set(clustery) not in our_clusters_as_sets:
    our_clusters_as_sets.append(set(clustery))
    our_clusters_new.append(clustery)
even_newer_clustering = []
sequence_list = []
for cluster in our_clusters_new:
  good_cluster = True
  for sequence in cluster:
    if sequence not in sequence_list:
      sequence_list.append(sequence)
    elif sequence in sequence_list:
      good_cluster = False
  if good_cluster:
    even_newer_clustering.append(cluster)


under_already_counted = {}
gamma = 1.0 # as gamma increases, the accuracy measurement becomes more strict
sum_of_indicators = 0
# we will compare the given underlying cluster to the given cluster of our clustering
for our_cluster in even_newer_clustering:
    for underlying_cluster in underlying:
        if all(sequence in underlying_cluster for sequence in our_cluster):
            # check if the intersection is big enough
            # intersection = [value for value in underlying_cluster if value in our_cluster]
            if len(our_cluster) >= gamma * len(underlying_cluster):
                sum_of_indicators += 1

# after you have gone through every cluster in our clustering, calculate accuracy
accuracy = sum_of_indicators / len(underlying)
print(accuracy)
# how many clusters are in our underlying clustering?
print(f'length of underlying is {len(underlying)}')
# how many clusters are in our clustering?
print('length of newer clustering', len(even_newer_clustering))

## Aligning the clusters

### Once again I have commented the rest of their code out, because it deals with decoding, which I never worked on.

As for the alignment stage, there are a number of options differing in accuracy and speed. For the sake of having the highest accuracy, we chose the Muscle function which is the most accurate multiple-alignment function among all python built-in functions.

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# import os
# from google.colab import drive

# # Mount Google Drive
# drive.mount('/content/drive')

# # Create a folder in the root directory
# !mkdir -p "/content/drive/My Drive/Fasta_Files"

In [ ]:
# !pip install virtualenv
# !virtualenv venv
# !source venv/bin/activate
# !pip install muscle3

In [ ]:
# !apt-get -qq install muscle

In [ ]:
# apt-get show muscle

In [ ]:
# def multiple_alignment_muscle(cluster,out=False):
#     # write cluster to file
#     # file = open("clm.fasta","w")
#     file = open('/content/drive/My Drive/Fasta_Files/clm.fasta', 'w')
#     print(cluster)
#     for i,c in enumerate(cluster):
#         file.write(">S%d\n" % i)
#         file.write(c)
#         file.write("\n")
#     file.close()
#     f = open("clm.fasta", "r")
#     stop = 0
#     for line in f:
#       print(line.strip())
#       stop+=1
#       if stop == 10:
#         break

#   # "~/muscle3.8.31_i86linux64"
#     # from google.colab import files
#     # uploaded = files.upload()
#     # muscle_exe2 = list(uploaded.keys())[0]
#     !chmod 755 "/content/drive/MyDrive/muscle5.1.linux_intel64"
#     muscle_exe = "/content/drive/MyDrive/muscle5.1.linux_intel64" # assuming you've already put this in the main directory
#     # /usr/local/lib/python3.10/dist-packages/muscle3
#     muscle_exe3 = "/usr/local/lib/python3.10/dist-packages/muscle3"
#     # with open(muscle_exe, 'rb') as f:
#     #   exe_content = f.read()
#     # print(exe_content)
#     # print(muscle_exe2)
#     print('+++++++++++++++++')
#     output_alignment = "clmout.fasta"
#     # I added this line below
#     import sys,os, subprocess
#     from io import StringIO
#     from Bio import AlignIO
#     from Bio.Align.Applications import MuscleCommandline
#     muscle_cline = MuscleCommandline(muscle_exe3, input="clm.fasta", out=output_alignment)
#     print(muscle_cline)
#     print('bruh')
#     print(type(muscle_cline))
#     #                             universal_newlines=True
#     # muscle_cline2 = r"muscle -align clm.fasta -output clmout.fasta"
#     # child = subprocess.Popen(muscle_cline2,
#     #                         stdout = subprocess.PIPE,
#     #                         stderr=subprocess.PIPE,
#     #                         shell=(sys.platform!="win32"),
#     #                         text=True)
#     # print(child)
#     print('yeet')
#     stdout, stderr = muscle_cline()
#     print('bug')
#     # print(child.stdout)
#     # print(type(child.stdout))
#     # with open(stdout, "r") as handle:
#     #     alignment = AlignIO.read(handle, "fasta")
#     msa = AlignIO.read(stdout, "fasta")
#     print('nuts')
#     if out:
#         print(msa)
#     alignedcluster = []
#     for i in msa:
#         alignedcluster += [i.seq]
#     return alignedcluster

In [ ]:
# fresults = []
# def align_clusters(clusters,masize = 15):
#     ### align clusters, generate candidates
#     for i, clusterinds in enumerate(clusters):
#         cluster = [reads[i] for i in clusterinds]
#         if len(cluster) < 3:
#             continue
#         if len(cluster) > masize:
#             for j in range(5):
#                 shuffle(cluster)
#                 ma = multiple_alignment_muscle(cluster[:masize])
#                 fresults.append(ma)
#         else:
#             ma = multiple_alignment_muscle(cluster[:masize])
#             fresults.append(ma)

#         if i % 1000 == 0:
#             print("%",round(i*100/len(clusters),2),"of the clusters are aligned.")

In [ ]:
# s = time.time()
# align_clusters(fclusts,15)
# print("Alignment Runtime:",round(time.time()-s,2),"s")

## Majority merging
At this stage, we merge each aligned cluster by putting up a voting for each position within the aligned sequences. Then, the fraction of all original sequences recovered is computed.

In [ ]:
# # This function returns the fraction of origignal squences recovered given a number of candidates
# def fraction_recovered(candidates,orig_seqs):
#     d = {}
#     for seq in orig_seqs:
#         d[seq] = 0
#     for cand in candidates:
#         if cand in d:
#             d[cand] += 1
#     av = sum([ d[seq]>0 for seq in d]) / len(d)
#     print("Fraction of recovered sequences: ", av )
#     if av>0:
#         print("Fraction of recovered sequences: ", sum([ d[seq] for seq in d]) / len(d) / av )

In [ ]:
# def majority_merge(reads,weight = 0.4):
#     # assume reads have the same length
#     res = ""
#     for i in range(len(reads[0])):
#         counts = {'A':0,'C':0,'G':0,'T':0,'-':0,'N':0}
#         for j in range(len(reads)):
#             counts[reads[j][i]] +=1
#         counts['-'] *= weight
#         mv = max(counts.items(), key=operator.itemgetter(1))[0]
#         if mv != '-':
#             res += mv
#     return res

In [ ]:
# candidates = []
# for ma in fresults:
#     candidates.append(majority_merge(ma,weight=0.5))
# fraction_recovered( [seq[:60] for seq in candidates] , orig_seqs)

In [ ]:
# f = open("./data/data_drawnseg.txt","w")
# for i,seq in enumerate(candidates):
#     if len(seq) >= 60:
#         f.write(seq[:60])
#         f.write('\n')
# f.close()

## Decoding

In [ ]:
# def decode(infile):
#     stat = sp.Popen(["./simulate/texttodna", "--decode",  "--n=16383", "--k=10977", "--N=20", "--K=18", "--nuss=6", "--l=4", \
#                     "--numblocks=1", infile, "--output=./data/data_rec.zip"],stdout=sp.PIPE)
#     for line in stat.stdout.read().splitlines():
#         print(line)
#     return True

In [ ]:
# s = time.time()
# decode("--input=./data/data_drawnseg.txt")
# runtime = round(time.time()-s,2)

In [ ]:
# print("Decoding time:",runtime,"s")

## Evaluation
Let's first extract the decoded file (data_rec.zip)

In [ ]:
# with zipfile.ZipFile('./data/data_rec.zip', 'r') as zipObj:
#    # Extract zip file contents to the "decoded" directory
#    zipObj.extractall(path = "./data/decoded")

By looking at the files in File_1.zip and the decoded files located in the 'decoded' folder, we see that the original files are completely recovered.